### Data Generation dataset preparation

### FIXME

1. Choose between default and custom dataset in FIXME 1 - default for the dataset used in this tutorial notebook; custom for a different dataset
1. Assign path of DATA_DIR in FIXME 2
1. Assign Cloud credentials in FIXME 3

We will use the https://www.kaggle.com/datasets/fantacher/neu-metal-surface-defects-data dataset to train a StyleGAN-XL model. This dataset consists of six classes of images, organized into a folder structure where each class corresponds to a subfolder containing its respective images. The hierarchy of the `neu-metal-surface-defects-data` dataset is as follows:

```
neu-metal-surface-defects-data/
├── class_1/
│   ├── class_1_image1.png
│   ├── class_1_image2.png
│   ├── ...
│   ├── class_1_imageN.png
├── class_2/
│   ├── class_2_image1.png
│   ├── class_2_image2.png
│   ├── ...
│   ├── class_2_imageN.png
```


In [ ]:
import os
import zipfile
import shutil

In [ ]:
dataset_to_be_used = "default" #FIXME1 
DATA_DIR = "/data/stylegan_xl" #FIXME2
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

### Dataset download and pre-processing <a class="anchor" id="head-1"></a>

In [ ]:
if dataset_to_be_used == "default":
  if not os.path.exists(os.path.join(DATA_DIR, "neu-metal-surface-defects-data.zip")):
    ! curl -L -o $DATA_DIR/neu-metal-surface-defects-data.zip\
      https://www.kaggle.com/api/v1/datasets/download/fantacher/neu-metal-surface-defects-data

In [ ]:
if dataset_to_be_used == "default":
    # Path to the ZIP file
    zip_file_path = os.path.join(DATA_DIR, "neu-metal-surface-defects-data.zip")

    # Directory to extract files
    extract_dir = DATA_DIR

    # Desired folder name
    desired_folder_name = "NEU_Metal_Surface_Defects_Data"

    # Extract the ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # Original extracted folder name
    original_folder_name = os.path.join(extract_dir, "NEU Metal Surface Defects Data")

    # Rename the folder
    renamed_folder = os.path.join(extract_dir, desired_folder_name)
    if os.path.exists(renamed_folder):
        shutil.rmtree(renamed_folder)  # Remove if the target folder already exists
    os.rename(original_folder_name, renamed_folder)

    # Convert .bmp files to .png
    for root, dirs, files in os.walk(renamed_folder):
        for file in files:
            if file.endswith(".bmp"):
                bmp_file_path = os.path.join(root, file)
                png_file_path = os.path.join(root, file.replace(".bmp", ".png"))

                # Convert BMP to PNG
                !python3 -m pip install Pillow
                from PIL import Image
                with Image.open(bmp_file_path) as img:
                    img.convert("RGB").save(png_file_path, "PNG")

                # Remove the original BMP file
                os.remove(bmp_file_path)

    print(f"Extraction and conversion completed. Files are in: {renamed_folder}")

In [ ]:
! ls -all $DATA_DIR/NEU_Metal_Surface_Defects_Data

### Create Tar files to upload

In [ ]:
!mkdir -p $DATA_DIR/cloud_folders/data/styleganxl_train $DATA_DIR/cloud_folders/data/styleganxl_val $DATA_DIR/cloud_folders/data/styleganxl_test

!mkdir -p $DATA_DIR/NEU_Metal_Surface_Defects_Data/train_images/images
!mkdir -p $DATA_DIR/NEU_Metal_Surface_Defects_Data/valid_images/images
!mkdir -p $DATA_DIR/NEU_Metal_Surface_Defects_Data/test_images/images

!cp -r $DATA_DIR/NEU_Metal_Surface_Defects_Data/train/* $DATA_DIR/NEU_Metal_Surface_Defects_Data/train_images/images
!cp -r $DATA_DIR/NEU_Metal_Surface_Defects_Data/valid/* $DATA_DIR/NEU_Metal_Surface_Defects_Data/valid_images/images
!cp -r $DATA_DIR/NEU_Metal_Surface_Defects_Data/test/* $DATA_DIR/NEU_Metal_Surface_Defects_Data/test_images/images

!tar -C $DATA_DIR/NEU_Metal_Surface_Defects_Data/train_images/ -czf $DATA_DIR/cloud_folders/data/styleganxl_train/images.tar.gz images
!tar -C $DATA_DIR/NEU_Metal_Surface_Defects_Data/valid_images/ -czf $DATA_DIR/cloud_folders/data/styleganxl_val/images.tar.gz images
!tar -C $DATA_DIR/NEU_Metal_Surface_Defects_Data/test_images/ -czf $DATA_DIR/cloud_folders/data/styleganxl_test/images.tar.gz images

### Final step: Upload the /data folder to your cloud storage and move on to running the API requests example notebooks
When you do a ls of your bucket it should have /data folder and the subfolders we created above within in (classification_train, classification_val, classification_test)

In [ ]:
!python3 -m pip install --upgrade awscli
ACCESS_KEY=FIXME3.1
SECRET_KEY=FIXME3.2
BUCKET_NAME=FIXME3.3
!AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp {DATA_DIR}/cloud_folders/data/stylegan_train s3://{BUCKET_NAME}/data/stylegan_train --recursive
!AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp {DATA_DIR}/cloud_folders/data/stylegan_val s3://{BUCKET_NAME}/data/stylegan_val --recursive
!AWS_ACCESS_KEY_ID={ACCESS_KEY} AWS_SECRET_ACCESS_KEY={SECRET_KEY} aws s3 cp {DATA_DIR}/cloud_folders/data/stylegan_test s3://{BUCKET_NAME}/data/stylegan_test --recursive